In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os

In [10]:
from utils.config import Config
config_file = "configs/config.json"
cfg = Config.from_file(config_file)

In [7]:
from models.videochat import VideoChat
from utils.easydict import EasyDict
import torch

from transformers import StoppingCriteria, StoppingCriteriaList

from PIL import Image
import numpy as np
import numpy as np
from decord import VideoReader, cpu
import torchvision.transforms as T
from models.video_transformers import (
    GroupNormalize, GroupScale, GroupCenterCrop, 
    Stack, ToTorchFormatTensor
)
from torchvision.transforms.functional import InterpolationMode

from torchvision import transforms

/nvme/likunchang/miniconda3/envs/chatcap/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
model = VideoChat(config=cfg.model)

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:18<00:00,  6.20s/it]


In [11]:
model = model.to(torch.device(cfg.device))
model = model.eval()

In [12]:
def get_prompt(conv):
    ret = conv.system + conv.sep
    for role, message in conv.messages:
        if message:
            ret += role + ": " + message + conv.sep
        else:
            ret += role + ":"
    return ret


def get_context_emb(conv, model, img_list):
    prompt = get_prompt(conv)
    print(prompt)
    if '<VideoHere>' in prompt:
        prompt_segs = prompt.split('<VideoHere>')
    else:
        prompt_segs = prompt.split('<ImageHere>')
    assert len(prompt_segs) == len(img_list) + 1, "Unmatched numbers of image placeholders and images."
    seg_tokens = [
        model.llama_tokenizer(
            seg, return_tensors="pt", add_special_tokens=i == 0).to("cuda:0").input_ids
        # only add bos to the first seg
        for i, seg in enumerate(prompt_segs)
    ]
    seg_embs = [model.llama_model.model.embed_tokens(seg_t) for seg_t in seg_tokens]
    mixed_embs = [emb for pair in zip(seg_embs[:-1], img_list) for emb in pair] + [seg_embs[-1]]
    mixed_embs = torch.cat(mixed_embs, dim=1)
    return mixed_embs


def ask(text, conv):
    conv.messages.append([conv.roles[0], text + '\n'])
        

class StoppingCriteriaSub(StoppingCriteria):
    def __init__(self, stops=[], encounters=1):
        super().__init__()
        self.stops = stops
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(stop):])).item():
                return True
        return False
    
    
def answer(conv, model, img_list, max_new_tokens=200, num_beams=1, min_length=1, top_p=0.9,
               repetition_penalty=1.0, length_penalty=1, temperature=1.0):
    stop_words_ids = [
        torch.tensor([835]).to("cuda:0"),
        torch.tensor([2277, 29937]).to("cuda:0")]  # '###' can be encoded in two different ways.
    stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])
    
    conv.messages.append([conv.roles[1], None])
    embs = get_context_emb(conv, model, img_list)
    outputs = model.llama_model.generate(
        inputs_embeds=embs,
        max_new_tokens=max_new_tokens,
        stopping_criteria=stopping_criteria,
        num_beams=num_beams,
        do_sample=True,
        min_length=min_length,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        length_penalty=length_penalty,
        temperature=temperature,
    )
    output_token = outputs[0]
    if output_token[0] == 0:  # the model might output a unknow token <unk> at the beginning. remove it
            output_token = output_token[1:]
    if output_token[0] == 1:  # some users find that there is a start token <s> at the beginning. remove it
            output_token = output_token[1:]
    output_text = model.llama_tokenizer.decode(output_token, add_special_tokens=False)
    output_text = output_text.split('###')[0]  # remove the stop sign '###'
    output_text = output_text.split('Assistant:')[-1].strip()
    conv.messages[-1][1] = output_text
    return output_text, output_token.cpu().numpy()

In [15]:
def get_index(num_frames, num_segments):
    seg_size = float(num_frames - 1) / num_segments
    start = int(seg_size / 2)
    offsets = np.array([
        start + int(np.round(seg_size * idx)) for idx in range(num_segments)
    ])
    return offsets


def load_video(video_path, num_segments=8, return_msg=False):
    vr = VideoReader(video_path, ctx=cpu(0))
    num_frames = len(vr)
    frame_indices = get_index(num_frames, num_segments)

    # transform
    crop_size = 224
    scale_size = 224
    input_mean = [0.48145466, 0.4578275, 0.40821073]
    input_std = [0.26862954, 0.26130258, 0.27577711]

    transform = T.Compose([
        GroupScale(int(scale_size), interpolation=InterpolationMode.BICUBIC),
        GroupCenterCrop(crop_size),
        Stack(),
        ToTorchFormatTensor(),
        GroupNormalize(input_mean, input_std) 
    ])

    images_group = list()
    for frame_index in frame_indices:
        img = Image.fromarray(vr[frame_index].asnumpy())
        images_group.append(img)
    torch_imgs = transform(images_group)
    if return_msg:
        fps = float(vr.get_avg_fps())
        sec = ", ".join([str(round(f / fps, 1)) for f in frame_indices])
        # " " should be added in the start and end
        msg = f"The video contains {len(frame_indices)} frames sampled at {sec} seconds."
        return torch_imgs, msg
    else:
        return torch_imgs

In [27]:
# vid_path = "./example/yoga.mp4"
vid_path = "./example/jesse_dance.mp4"
vid, msg = load_video(vid_path, num_segments=8, return_msg=True)
print(msg)
    
# The model expects inputs of shape: T x C x H x W
TC, H, W = vid.shape
video = vid.reshape(1, TC//3, 3, H, W).to("cuda:0")

img_list = []
image_emb, _ = model.encode_img(video)
img_list.append(image_emb)

The video contains 8 frames sampled at 0.6, 1.8, 3.0, 4.2, 5.5, 6.7, 7.9, 9.1 seconds.


In [28]:
chat = EasyDict({
#     "system": "You are an AI assistant. A human gives an image or a video and asks some questions. You should give helpful, detailed, and polite answers.\n",
    "system": "",
    "roles": ("Human", "Assistant"),
    "messages": [],
    "sep": "###"
})

chat.messages.append([chat.roles[0], f"<Video><VideoHere></Video> {msg}\n"])
# ask("Describe the video in detail.", chat)
# ask("Is she safe to doing something in the video?", chat)
# ask("Where is she?", chat)
# ask("Who are you?", chat)
# ask("Are you an assistant?", chat)
# ask("How can I learn to do as the video?", chat)
# ask("Tell me what she did at the fourth second", chat)
# ask("Can you provide me some urls to learn to do as the video?", chat)
# ask("List the president of America.", chat)
# ask("What do you feel from the video?", chat)
# ask("Do you think it is funny? Why?", chat)
ask("Explain why it is funny?", chat)
# ask("Explain why the video is ridiculous?", chat)

llm_message = answer(conv=chat, model=model, img_list=img_list, max_new_tokens=1000)[0]
print(llm_message)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


###Human: <Video><VideoHere></Video> The video contains 8 frames sampled at 0.6, 1.8, 3.0, 4.2, 5.5, 6.7, 7.9, 9.1 seconds.
###Human: Explain why it is funny?
###Assistant:
This is a funny video of a man dancing in a kitchen with his handy family of paper people. The man is clearly having fun, and the fact that the paper people are also moving and dancing alongside him adds to the humor of the video. The man's exuberance and enthusiasm while dancing with the paper people makes the video particularly entertaining. It is a lighthearted and whimsical moment that showcases a man's playful side. The use of paper people adds a touch of creativity and imagination to the video. Overall, it is a funny and heartwarming video.


In [24]:
# img_path = "./example/bear.jpg"
# img_path = "./example/cxk.png"
img_path = "./example/dog.png"
img = Image.open(img_path).convert('RGB')

transform = transforms.Compose(
    [
        transforms.Resize(
            (224, 224), interpolation=InterpolationMode.BICUBIC
        ),
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
    ]
)

img = transform(img).unsqueeze(0).unsqueeze(0).cuda()
img_list = []
image_emb, _ = model.encode_img(img)
img_list.append(image_emb)

In [26]:
chat = EasyDict({
#     "system": "You are an AI assistant. A human gives an Image and asks questions. You should give helpful, detailed, and polite answers.\n",
    "system": "",
    "roles": ("Human", "Assistant"),
    "messages": [],
    "sep": "###"
})

chat.messages.append([chat.roles[0], "<Image><ImageHere></Image>\n"])
# ask("Describe the image in detail.", chat)
# ask("What's in the image?", chat)
# ask("Who are you?", chat)
# ask("Are you an assistant?", chat) 
# ask("Where can I find the animal?", chat)
# ask("Give me some urls to find the anmimal", chat)
# ask("Give me some urls to learn to do it", chat)
# ask("Is it funny? why?", chat)
# ask("What do you feel from the image?", chat)
# ask("Do you think the image is funny?", chat)
ask("Explain why the image is funny?", chat)
# ask("What is the text in the image?", chat)

llm_message = answer(conv=chat, model=model, img_list=img_list, max_new_tokens=1000)[0]
print(llm_message)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


###Human: <Image><ImageHere></Image>
###Human: Explain why the image is funny?
###Assistant:
The image is funny because the puppy is sleeping on a wooden floor with the words, "just monday" written underneath it. This implies that the puppy is lazy and is using this as an excuse for not having to do anything else on Monday. The words, "just monday," are written in a bold font, emphasizing the puppy 's laziness and disdain for Monday. The puppy is lying down on the floor, which adds to the humor and implies that it doesn't want to move or do anything on this day of the week. The photo is taken from a slightly upward angle, which adds to the visual impact and creates a comical effect. The caption is written in a playful, humorous font that reinforces the joke and encourages viewers to have a good laugh. The combination of the puppy, the words, and the photo make this image funny and entertaining.
